In [1]:
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.utils import data
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os
import json
from pathlib import Path
import models.HDF5Dataset as H

import models.dcgan as dcgan
import models.mlp as mlp

In [2]:
cudnn.benchmark = True
if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")


## load data 
dataset = H.HDF5Dataset('/home/eren/CaloGAN/data/test/', recursive=False, load_data=False, data_cache_size=4, transform=None)
loader_params = {'batch_size': 100, 'shuffle': True, 'num_workers': 6}
dataloader = data.DataLoader(dataset, **loader_params)

ngpu = int(1)
nz = int(100)
ngf = int(32)
ndf = int(32)
nc = int(5)
n_extra_layers = int(0)
imageSize = 16
batchSize = 10
lrD = 0.0005
lrG = 0.0005
beta1 = 0.5

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

            

In [3]:
netG = dcgan.DCGAN_G(imageSize, nz, nc, ngf, ngpu, n_extra_layers)
netG.apply(weights_init)
print(netG)

DCGAN_G(
  (main): Sequential(
    (initial:100-64:convt): ConvTranspose2d(100, 64, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (initial:64:relu): ReLU(inplace)
    (pyramid:64-32:convt): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid:32:batchnorm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid:32:relu): ReLU(inplace)
    (final:32-5:convt): ConvTranspose2d(32, 5, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (final:5:tanh): Tanh()
  )
)


In [6]:
netD = dcgan.DCGAN_D(imageSize, nz, nc, ndf, ngpu, n_extra_layers)
netD.apply(weights_init)

DCGAN_D(
  (main): Sequential(
    (initial:5-32:conv): Conv2d(5, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (initial:32:relu): LeakyReLU(negative_slope=0.2, inplace)
    (pyramid:32-64:conv): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid:64:relu): LeakyReLU(negative_slope=0.2, inplace)
    (final:64-1:conv): Conv2d(64, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

In [4]:
input = torch.FloatTensor(batchSize, nc, imageSize, imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

In [7]:
optimizerD = optim.Adam(netD.parameters(), lr=lrD, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lrG, betas=(beta1, 0.999))

In [ ]:
real_cpu[0][0].size(0)

In [ ]:
t = real_cpu[0][0].view(real_cpu[0][0].size(0), real_cpu[0][0].size(1), real_cpu[0][0].size(2))

In [ ]:
t.shape

In [ ]:
input.shape

In [ ]:
for x,y in dataloader:
    print (x[0][0].shape, x[1][0].shape, y[0].shape)